# Maximizing National Welfare Under Price Controls: A Tariff Optimization Model
### MAT 4800: Introduction to Nonlinear Optimization

In [1]:
import numpy as np
from scipy.optimize import minimize

# -------------------------
# Shared Functions
# -------------------------
def D(p): return b_d - m_d * p ** a_d
def S(p): return b_s + m_s * p ** a_s
def pe(t, p_i): return p_i / (t + 1)

# -------------------------
# Welfare Objective
# -------------------------
def welfare(x):
    t, p_i = x
    p_e = pe(t, p_i)
    return (
        (p_f - p_e) * (D(p_i) - S(p_i))
        - 0.5 * (p_f - p_e) * (S(p_i) - S(p_e))
        - 0.5 * (p_f - p_e) * (D(p_e) - D(p_i))
    )

def welfare2(x):
    t, p_i = x
    p_e = pe(t, p_i)
    exporter_loss = abs((S(p_f) - D(p_f)) + (S(p_e) - D(p_e)))
    return (
        (p_f - p_e) * (D(p_i) - S(p_i))
        - 0.5 * (p_f - p_e) * (S(p_i) - S(p_e))
        - 0.5 * (p_f - p_e) * (D(p_e) - D(p_i))
        - 0.5 * (p_f - p_e) * exporter_loss
    )

## Steel Import Optimization

In [2]:
# -------------------------
# Parameters
# -------------------------
p_f = 883  # Free trade price

b_d = 8            # Demand intercept
m_d = 0.001716     # Demand slope
a_d = 1.0          # Demand curvature

b_s = 1            # Supply intercept
m_s = 0.000189     # Supply slope
a_s = 1.0          # Supply curvature

t_s = 0                 # Tariff floor
p_s = -0.2 * p_f + p_f  # Price floor
t_c = 1                 # Tariff cap
p_c = 0.2 * p_f + p_f   # Price cap

# -------------------------
# Optimization Setup
# -------------------------
x0 = [0.1, 850]  # Initial guess
bounds = [(t_s, t_c), (p_s, p_c)]

# -------------------------
# Optimization Without Exporter Loss
# -------------------------
print("--- WITHOUT Exporter Loss ---")
res1 = minimize(lambda x: -welfare(x), x0, bounds=bounds, method="trust-constr")
t1, p1 = res1.x
W1_val = welfare([t1, p1])
imports1 = D(p1) - S(p1)

print(f"Optimal tariff t: {t1:.4f}")
print(f"Optimal price p_i: {p1:.4f}")
print(f"Actual welfare: {W1_val:.4f} million USD")
print(f"Import quantity (D - S): {imports1:.4f} million tons\n")

# -------------------------
# Optimization With Exporter Loss
# -------------------------
print("--- WITH Exporter Loss ---")
res2 = minimize(lambda x: -welfare2(x), x0, bounds=bounds, method="trust-constr")
t2, p2 = res2.x
p_e2 = pe(t2, p2)
W2_val = welfare2([t2, p2])
imports2 = D(p2) - S(p2)

print(f"Optimal tariff t: {t2:.4f}")
print(f"Optimal price p_i: {p2:.4f}")
print(f"Actual welfare (including exporter loss): {W2_val:.4f} million USD")
print(f"Import quantity (D - S): {imports2:.4f} million tons")

--- WITHOUT Exporter Loss ---
Optimal tariff t: 0.9999
Optimal price p_i: 828.4490
Actual welfare: 2356.5556 million USD
Import quantity (D - S): 5.4218 million tons

--- WITH Exporter Loss ---
Optimal tariff t: 0.0012
Optimal price p_i: 849.9756
Actual welfare (including exporter loss): 1.0055 million USD
Import quantity (D - S): 5.3808 million tons


## More Elastic Demand

In [3]:
# -------------------------
# Parameters
# -------------------------
p_f = 883

# Elastic Demand function
b_d = 10             # Higher demand intercept
m_d = 0.005          # More elastic demand (steeper drop)
a_d = 1.0            # Still linear

# Supply function
b_s = 1              # Same supply intercept
m_s = 0.0001         # Flatter supply (slow to ramp up)
a_s = 1.0

# Tariff and price bounds
t_s = 0
p_s = 0.8 * p_f
t_c = 1
p_c = 1.2 * p_f

# -------------------------
# Optimization
# -------------------------
x0 = [0.1, 850]
bounds = [(t_s, t_c), (p_s, p_c)]

res = minimize(lambda x: -welfare(x), x0, bounds=bounds, method="trust-constr")
t_opt, p_opt = res.x
welfare_val = welfare([t_opt, p_opt])
imports = D(p_opt) - S(p_opt)

# -------------------------
# Output
# -------------------------
print("--- ELASTIC DEMAND EXAMPLE ---")
print(f"Optimal tariff t: {t_opt:.4f}")
print(f"Optimal price p_i: {p_opt:.2f}")
print(f"Welfare: {welfare_val:.2f} million USD")
print(f"Imports: {imports:.2f} million tons")

--- ELASTIC DEMAND EXAMPLE ---
Optimal tariff t: 0.9998
Optimal price p_i: 846.96
Welfare: 1654.47 million USD
Imports: 4.68 million tons


## Supply-Constrained Market

In [ ]:
# -------------------------
# Parameters
# -------------------------
p_f = 883

# Demand function
b_d = 8
m_d = 0.001716
a_d = 1.0

# Supply-constrained market (very inelastic - slow to increase)
b_s = 1
m_s = 0.00001  # Very inelastic supply
a_s = 1.0

# Tariff and price bounds
t_s = 0
p_s = 0.8 * p_f
t_c = 1
p_c = 1.2 * p_f

# -------------------------
# Optimization
# -------------------------
x0 = [0.1, 850]
bounds = [(t_s, t_c), (p_s, p_c)]

res = minimize(lambda x: -welfare(x), x0, bounds=bounds, method="trust-constr")
t_opt, p_opt = res.x
welfare_val = welfare([t_opt, p_opt])
imports = D(p_opt) - S(p_opt)

# -------------------------
# Output
# -------------------------
print("--- SUPPLY-CONSTRAINED EXAMPLE ---")
print(f"Optimal tariff t: {t_opt:.4f}")
print(f"Optimal price p_i: {p_opt:.2f}")
print(f"Welfare: {welfare_val:.2f} million USD")
print(f"Imports: {imports:.4f} million tons")

--- SUPPLY-CONSTRAINED EXAMPLE ---
Optimal tariff t: 0.9999
Optimal price p_i: 825.97
Welfare: 2452.43 million USD
Imports: 5.5744 million tons
